In [115]:
import numpy as np
import cv2

import PIL.Image as image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

### mobilenet is a pretrained model that is trained on 1.4 million images and 1000 different classes

- images are in 224x224 dimension

In [116]:
IMAGE_SHAPE = (224,224)
model_224 = MobileNetV2(
    include_top=True,
    input_shape=IMAGE_SHAPE+(3,),
    weights='imagenet'
)
# model_224.trainable = False

In [117]:
image_lables = []
with open("../ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [118]:
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [119]:
import pathlib
data_dir = pathlib.Path('../datasets/flower_photos')
data_dir

PosixPath('../datasets/flower_photos')

In [120]:
flowers_images_dict = {
    'roses' : list(data_dir.glob("roses/*")),
    'tulips' : list(data_dir.glob("tulips/*")),
    # 'daisy' : list(data_dir.glob("daisy/*")),
    # 'dandelion' : list(data_dir.glob("dandelion/*")),
    # 'sunflowers' : list(data_dir.glob("sunflowers/*")),
}

In [121]:
flowers_label_dict = {
    'roses' : 0,
    'tulips' : 1,
    # 'daisy' : 2,
    # 'dandelion' : 3,
    # 'sunflowers' : 4,
}

In [122]:
str(flowers_images_dict['roses'][0])

'../datasets/flower_photos/roses/16209331331_343c899d38.jpg'

In [123]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img.shape

(243, 500, 3)

### before training model we need to make sure all images are of same size
- here we should reshape image to 224x224

In [124]:
X, y = [], []

for flower_name, images in flowers_images_dict.items() :
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(flowers_label_dict[flower_name])

### we should normalize the image by dividing it by 255 to bring the value to 0 to 1

In [125]:
X = np.array(X)
y = np.array(y)

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [127]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [128]:
predicted = model_224.predict(np.array([X[0], X[1], X[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

array([794, 794, 721])

In [129]:
y[3]

0

In [130]:
image_labels[791]

'shopping basket'

## now lets import mobilenet model without the last layer 

In [131]:
model_224_without_top_layer = MobileNetV2(
    include_top=False,
    input_shape=IMAGE_SHAPE+(3,),
    weights = 'imagenet'
)
model_224_without_top_layer.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(model_224_without_top_layer.layers))

# Fine-tune from this layer onwards
fine_tune_from_layer_number = 152

# Freeze all the layers before the `fine_tune_from_layer_number` layer
for layer in model_224_without_top_layer.layers[:fine_tune_from_layer_number]:
  layer.trainable = False

model_224_without_top_layer.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['acc']
)

model_224_without_top_layer.summary()

Number of layers in the base model:  154
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )     

In [132]:
num_of_flowers = 2
model = tf.keras.Sequential([
    model_224_without_top_layer,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_16  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_14 (Dense)            (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 5,122
Non-trainable params: 2,255,424
_________________________________________________________________


In [133]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['acc']
)

In [134]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_16  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_14 (Dense)            (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 5,122
Non-trainable params: 2,255,424
_________________________________________________________________


In [135]:
model.fit(X_train_scaled, y_train, epochs=15)

Epoch 1/15
34/34 [==============================] - 42s 925ms/step - loss: 0.7378 - acc: 0.5602
Epoch 2/15
34/34 [==============================] - 33s 949ms/step - loss: 0.6925 - acc: 0.5574
Epoch 3/15
34/34 [==============================] - 31s 903ms/step - loss: 0.6925 - acc: 0.5574
Epoch 4/15
34/34 [==============================] - 31s 886ms/step - loss: 0.6925 - acc: 0.5574
Epoch 5/15
34/34 [==============================] - 30s 884ms/step - loss: 0.6925 - acc: 0.5574
Epoch 6/15
34/34 [==============================] - 31s 915ms/step - loss: 0.6919 - acc: 0.5574
Epoch 7/15
34/34 [==============================] - 30s 877ms/step - loss: 0.6919 - acc: 0.5574
Epoch 8/15
34/34 [==============================] - 30s 882ms/step - loss: 0.6925 - acc: 0.5574
Epoch 9/15
34/34 [==============================] - 30s 869ms/step - loss: 0.6919 - acc: 0.5574
Epoch 10/15
34/34 [==============================] - 31s 901ms/step - loss: 0.6925 - acc: 0.5574
Epoch 11/15
34/34 [====================

In [138]:
model.evaluate(X_test_scaled, y_test)

12/12 [==============================] - 8s 627ms/step - loss: 0.6931 - acc: 0.5472


[0.6931471824645996, 0.5472221970558167]

In [137]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,